In [1]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer,AutoModelForSequenceClassification, DataCollatorWithPadding, AdamW
import torch
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
raw_datasets = load_dataset("khondoker/SentNoB")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/khondoker--SentNoB-1d4e2e75659b8316/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_

  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Data', 'Label'],
        num_rows: 15728
    })
})

In [4]:
train_test_valid = raw_datasets['train'].train_test_split(test_size = 0.2, shuffle=True, seed = 42)

In [5]:
train_test_valid

DatasetDict({
    train: Dataset({
        features: ['Data', 'Label'],
        num_rows: 12582
    })
    test: Dataset({
        features: ['Data', 'Label'],
        num_rows: 3146
    })
})

In [6]:
test_valid = train_test_valid['test'].train_test_split(test_size=0.5, shuffle=True, seed = 42)

In [7]:
test_valid

DatasetDict({
    train: Dataset({
        features: ['Data', 'Label'],
        num_rows: 1573
    })
    test: Dataset({
        features: ['Data', 'Label'],
        num_rows: 1573
    })
})

In [8]:
raw_datasets = DatasetDict({
    'train': train_test_valid['train'],
    'validation':test_valid['train'],
    'test':test_valid['test']
})

In [9]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Data', 'Label'],
        num_rows: 12582
    })
    validation: Dataset({
        features: ['Data', 'Label'],
        num_rows: 1573
    })
    test: Dataset({
        features: ['Data', 'Label'],
        num_rows: 1573
    })
})

In [10]:
raw_datasets = raw_datasets.rename_column("Label","labels")
raw_datasets = raw_datasets.rename_column("Data","text")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 12582
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 1573
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 1573
    })
})

In [11]:
checkpoint = 'csebuetnlp/banglabert'

In [12]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 3)

Some weights of the model checkpoint at csebuetnlp/banglabert were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.bias', 'classifier.

In [13]:
def tokenize_fn(examples):
    return tokenizer(examples['text'], truncation = 'longest_first')

In [14]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 12582
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 1573
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 1573
    })
})

In [15]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched = True)

  0%|          | 0/13 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [16]:
tokenized_datasets


DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12582
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1573
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1573
    })
})

In [17]:
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12582
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1573
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1573
    })
})

In [18]:
tokenized_datasets.set_format('torch')

In [19]:
data_collator = DataCollatorWithPadding(tokenizer)

In [20]:
from torch.utils.data import DataLoader

In [21]:
train_dataloader = DataLoader(tokenized_datasets['train'], shuffle = True, batch_size = 8, collate_fn = data_collator)
train_dataloader

In [22]:
eval_dataloader = DataLoader(tokenized_datasets['validation'], shuffle = True , batch_size = 8, collate_fn= data_collator)
eval_dataloader

In [23]:
test_dataloader = DataLoader(tokenized_datasets['test'], shuffle = True , batch_size = 8, collate_fn= data_collator)
test_dataloader

In [24]:
flag = 0
data_dict=[]
for i in train_dataloader:
    if flag > 3:
        break
    
    data_dict.append([{k:v.shape for k,v in i.items()}])
    flag = flag + 1

data_dict

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[[{'labels': torch.Size([8]),
   'input_ids': torch.Size([8, 50]),
   'token_type_ids': torch.Size([8, 50]),
   'attention_mask': torch.Size([8, 50])}],
 [{'labels': torch.Size([8]),
   'input_ids': torch.Size([8, 28]),
   'token_type_ids': torch.Size([8, 28]),
   'attention_mask': torch.Size([8, 28])}],
 [{'labels': torch.Size([8]),
   'input_ids': torch.Size([8, 58]),
   'token_type_ids': torch.Size([8, 58]),
   'attention_mask': torch.Size([8, 58])}],
 [{'labels': torch.Size([8]),
   'input_ids': torch.Size([8, 49]),
   'token_type_ids': torch.Size([8, 49]),
   'attention_mask': torch.Size([8, 49])}]]

In [25]:
outputs = model(**i)

In [26]:
print(outputs.loss)
print(outputs.logits.shape)
print(outputs.logits)

tensor(1.0490, grad_fn=<NllLossBackward0>)
torch.Size([8, 3])
tensor([[-0.0784, -0.0219,  0.0757],
        [-0.0769, -0.0238,  0.0695],
        [-0.0451,  0.0336, -0.0126],
        [-0.0699,  0.0038,  0.0776],
        [-0.0553,  0.0172,  0.0681],
        [-0.0680,  0.0009,  0.0426],
        [-0.0920,  0.0100,  0.0367],
        [-0.0705,  0.0076,  0.0553]], grad_fn=<AddmmBackward0>)


In [27]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [28]:
from transformers import get_scheduler

In [29]:
len(train_dataloader)

1573

In [30]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
num_training_steps

4719

In [31]:
lr_scheduler = get_scheduler("linear", optimizer = optimizer, num_warmup_steps = 0, num_training_steps = num_training_steps)

In [32]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device('cpu')

In [33]:
model.to(device)
print(device)

cuda


In [34]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k:v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/4719 [00:00<?, ?it/s]

In [35]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [36]:
import evaluate

In [37]:
def compute_metrics(eval_preds):
    metric = evaluate.combine(["accuracy"])
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [38]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.7221869040050858}

In [39]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.7476160203432931}

In [40]:
predictions = []
ground_truth_labels = []

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    ##print(batch)
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    batch_predictions = logits.argmax(dim=-1).tolist()
    predictions.extend(batch_predictions)
    ground_truth_labels.extend(batch['labels'].tolist())
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

accuracy = accuracy_score(ground_truth_labels, predictions)
f1_macro = f1_score(ground_truth_labels, predictions, average='macro')
f1_micro = f1_score(ground_truth_labels, predictions, average='micro')

precision = precision_score(ground_truth_labels, predictions, average="weighted")
recall = recall_score(ground_truth_labels, predictions, average="weighted")
class_report = classification_report(ground_truth_labels, predictions)
conf_matrix = confusion_matrix(ground_truth_labels, predictions)

print("Accuracy:", accuracy)
print("F1 Macro:", f1_macro)
print("F1 Micro:", f1_micro)
print('Precision:', precision)
print("Recall:", recall)
print("Classification Report:\n", class_report)
print("Confusion Matrix: \n", conf_matrix)


Accuracy: 0.7221869040050858
F1 Macro: 0.6994133806910036
F1 Micro: 0.7221869040050858
Precision: 0.7248110476290065
Recall: 0.7221869040050858
Classification Report:
               precision    recall  f1-score   support

           0       0.52      0.54      0.53       373
           1       0.78      0.78      0.78       654
           2       0.80      0.77      0.78       546

    accuracy                           0.72      1573
   macro avg       0.70      0.70      0.70      1573
weighted avg       0.72      0.72      0.72      1573

Confusion Matrix: 
 [[203 101  69]
 [102 513  39]
 [ 83  43 420]]


In [41]:
predictions = []
ground_truth_labels = []

for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    ##print(batch)
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    batch_predictions = logits.argmax(dim=-1).tolist()
    predictions.extend(batch_predictions)
    ground_truth_labels.extend(batch['labels'].tolist())
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

accuracy = accuracy_score(ground_truth_labels, predictions)
f1_macro = f1_score(ground_truth_labels, predictions, average='macro')
f1_micro = f1_score(ground_truth_labels, predictions, average='micro')

precision = precision_score(ground_truth_labels, predictions, average="weighted")
recall = recall_score(ground_truth_labels, predictions, average="weighted")
class_report = classification_report(ground_truth_labels, predictions)
conf_matrix = confusion_matrix(ground_truth_labels, predictions)

print("Accuracy:", accuracy)
print("F1 Macro:", f1_macro)
print("F1 Micro:", f1_micro)
print('Precision:', precision)
print("Recall:", recall)
print("Classification Report:\n", class_report)
print("Confusion Matrix: \n", conf_matrix)


Accuracy: 0.7476160203432931
F1 Macro: 0.7136513908244271
F1 Micro: 0.7476160203432932
Precision: 0.7448611565370576
Recall: 0.7476160203432931
Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.52      0.53       334
           1       0.79      0.81      0.80       668
           2       0.81      0.81      0.81       571

    accuracy                           0.75      1573
   macro avg       0.72      0.71      0.71      1573
weighted avg       0.74      0.75      0.75      1573

Confusion Matrix: 
 [[173 101  60]
 [ 81 539  48]
 [ 62  45 464]]
